### Installation 

In [ ]:
! pip install datasets transformers rouge-score nltk
!git clone https://github.com/xmu-xiaoma666/External-Attention-pytorch.git ext_attns

### About dataset 

In [4]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 11334
    })
})

In [6]:
print(f'train: {len(tokenized_datasets["train"])}')
print(f'val:   {len(tokenized_datasets["validation"])}')
print(f'test:  {len(tokenized_datasets["test"])}')

train: 204045
val:   11332
test:  11334


### 2L Attentions

|i|attention | used|
|---|--------| ----|
|0  |UFOAttention| ✔|
|1  | AFT_FULL   |x|
|2  | MUSEAttention|✔ |
|3  | EMSA|     x|
|4  |SimplifiedScaledDotProductAttention|✔ |
|5  |ScaledDotProductAttention|✔ |
|6  |ExternalAttention|✔ |

In [1]:
import os
ext_path = os.path.abspath('./ext_attns/')
import sys
sys.path.append(ext_path)
# list
from model.attention.UFOAttention import *
from model.attention.AFT import AFT_FULL
from model.attention.MUSEAttention import MUSEAttention
from model.attention.EMSA import EMSA
from model.attention.SimplifiedSelfAttention import SimplifiedScaledDotProductAttention
from model.attention.SelfAttention import ScaledDotProductAttention
from model.attention.ExternalAttention import ExternalAttention
d_model = 768 
n_head  = 12  
ufo = UFOAttention(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head)
aft_full = AFT_FULL(d_model=d_model, n=n_head)
ma = MUSEAttention(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head)
emsa = EMSA(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head,H=n_head,W=n_head,ratio=2,apply_transform=True)
ssa = SimplifiedScaledDotProductAttention(d_model=d_model, h=n_head)
sa = ScaledDotProductAttention(d_model=d_model, d_k=d_model//n_head, d_v=d_model//n_head, h=n_head)
ea = ExternalAttention(d_model=d_model,S=8) #v

attn_li = [ufo,aft_full,ma,emsa,ssa,sa,ea]

import transformers

# print(transformers.__version__)

model_checkpoint = "t5-small"
model_checkpoint = 'facebook/bart-base'

from datasets import load_dataset, load_metric

raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")

# raw_datasets["train"][0]

import datasets

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# ?
# with tokenizer.as_target_tokenizer():
#     print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

batch_size = 3 #16

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Using custom data configuration default
Reusing dataset xsum (/home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-5d732b1c86657ea0.arrow
Loading cached processed dataset at /home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-38966a651c72a103.arrow
Loading cached processed dataset at /home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-d19c279816291ade.arrow


In [2]:
trvaltest = tokenized_datasets['train'].train_test_split(test_size=0.004,train_size=0.005) # 800,1080
# trvaltest = tokenized_datasets['train'].train_test_split(test_size=0.004,train_size=0.01) # 800,2080
# trvaltest = tokenized_datasets['train'].train_test_split(test_size=0.004,train_size=0.02) # 800,4080
tr = trvaltest['train']
valtest = trvaltest['test'].train_test_split(test_size=0.5,train_size=0.5) # 208, 208
# valtest = trvaltest['test'].train_test_split(test_size=0.5,train_size=0.5) # 408, 408
# valtest = trvaltest['test'].train_test_split(test_size=0.05,train_size=0.05)
val = valtest['train']
te = valtest['test']

del trvaltest
del valtest

Loading cached split indices for dataset at /home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-a37fd9f11dd44add.arrow and /home/yp/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499/cache-195107633ef23937.arrow


In [3]:
from torch import nn
evals = {}
for i,attn in enumerate(attn_li[:]):
    not_allowed = ('AFT_FULL','EMSA')
    print(attn)
    if attn.__str__().startswith(not_allowed):
        continue
    
    
    import os
    model_name = attn.__str__().split('(')[0]
    # model_name = model_checkpoint.split("/")[-1] + model_name
    saved_path = os.path.join('./output',model_name)
    
    args = Seq2SeqTrainingArguments(
        saved_path,
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=15,
        predict_with_generate=True,
        fp16=True,
        save_strategy='epoch',
        load_best_model_at_end=True,
    #     push_to_hub=True,
    )

    class tel2lin1(nn.Module):
        def __init__(self):
            super(tel2lin1,self).__init__()
            # self.tel = nn.TransformerEncoderLayer(d_model=768,nhead=3,batch_first=True)
            self.tel = attn
            self.lin = nn.Linear(in_features=768,out_features=50265, bias=False)
            
        def forward(self,x):
            if self.tel.__str__().startswith('ExternalAttention'):
                x = self.tel(x)
            else:
                x = self.tel(x,x,x)
            x = self.lin(x)
            return x
    model.lm_head = tel2lin1()

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tr,
        eval_dataset=val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    evals[f'{model_name}'] = trainer.evaluate(eval_dataset=te)

UFOAttention(
  (fc_q): Linear(in_features=768, out_features=768, bias=True)
  (fc_k): Linear(in_features=768, out_features=768, bias=True)
  (fc_v): Linear(in_features=768, out_features=768, bias=True)
  (fc_o): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2550
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,7.253441,0.000000,0.000000,0.000000,0.000000,5.911800
2,No log,6.886457,2.917300,0.219800,2.664200,2.669900,9.453400
3,7.255100,6.739688,4.875600,0.333200,4.294900,4.270600,10.210800
4,7.255100,6.627885,10.141200,0.946400,9.057300,9.035600,19.144600
5,7.255100,6.531991,9.306900,1.111700,8.489900,8.506700,20.000000
6,5.949700,6.498371,11.935600,1.636700,10.323700,10.276100,19.698500
7,5.949700,6.465492,11.428300,1.423800,9.882300,9.887100,16.500000
8,5.949700,6.432230,13.255900,1.666600,11.264200,11.252400,17.576000
9,5.559900,6.386285,11.311000,1.510300,9.847400,9.857600,14.951000
10,5.559900,6.381336,10.845100,1.428200,9.429900,9.442700,14.757400


/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 6
Saving model checkpoint to ./output/UFOAttention/checkpoint-170
Configuration saved in ./output/UFOAttention/checkpoint-170/config.json
Model weights saved in ./output/UFOAttention/checkpoint-170/pytorch_model.bin
tokenizer config file saved in ./output/UFOAttention/checkpoint-170/tokenizer_config.json
Special tokens file saved in ./output/U

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


ExternalAttention(
  (mk): Linear(in_features=768, out_features=8, bias=False)
  (mv): Linear(in_features=8, out_features=768, bias=False)
  (softmax): Softmax(dim=1)
)


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2550


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,10.813046,7.761000,0.000000,7.724500,7.530400,8.000000
2,No log,10.768181,10.789500,0.000000,10.688400,10.697100,11.000000
3,10.780100,10.683411,10.429300,0.000000,10.333300,10.432900,14.000000
4,10.780100,10.577523,8.146900,0.000000,8.115200,7.591400,11.000000
5,10.780100,10.462443,6.027400,0.000000,6.023500,6.038400,8.000000
6,10.497300,10.346169,6.027400,0.000000,6.023500,6.038400,8.000000
7,10.497300,10.234465,6.027400,0.000000,6.023500,6.038400,8.000000
8,10.497300,10.132950,0.000000,0.000000,0.000000,0.000000,5.000000
9,10.127700,10.041049,0.000000,0.000000,0.000000,0.000000,5.000000
10,10.127700,9.963591,0.000000,0.000000,0.000000,0.000000,5.000000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 6
Saving model checkpoint to ./output/ExternalAttention/checkpoint-170
Configuration saved in ./output/ExternalAttention/checkpoint-170/config.json
Model weights saved in ./output/ExternalAttention/checkpoint-170/pytorch_model.bin
tokenizer config file saved in ./output/ExternalAttention/checkpoint-170/tokenizer_config.json
Special tokens file saved in ./output/ExternalAttention/checkpoint-170/special_tokens_map.json
Deleting older checkpoint [output/ExternalAttention/checkpoint-7] due to args.save_total_limit
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was a

### Light 2L Attentions 

### 2L Attentions with Skip connection 

### 4L Attentions

### Etc

In [11]:
from torch import nn
import torch,copy
class cons(nn.Module):
    def __init__(self):
        super(cons,self).__init__()
        self.front = nn.Sequential(
            nn.Conv2d(1,32,3),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(32),
            nn.ReLU() ,#batchnorm 2d ?
            nn.Conv2d(32,16,2),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Flatten()
        )
        sple = torch.rand(1,1,28,28)
        front_out = self.front(sple).size()[-1]
        self.body = nn.Sequential(
            nn.Linear(front_out,256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(p=0.2),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            # nn.Linear(128,10),
        )
        self.medium = nn.Sequential(
            nn.Linear(128,64),
            nn.Linear(64,128)
        )
        self.macro = [copy.deepcopy(self.medium) for _ in range(3)]
        self.macro = nn.ModuleList(self.macro)
        
        self.fc = nn.Sequential(
            nn.Linear(128,10),
            nn.Softmax(dim=1)
        )
    def forward(self,x):
        x = self.front(x)
        x = self.body(x)
        for i in range(len(self.macro)):
            x = self.macro[i](x)
        x = self.fc(x)
        return x
md = cons()

/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [2]:
from torch import nn
sli = [nn.Sequential(nn.Linear(128,64),nn.Linear(64,128)),
       nn.Sequential(nn.Linear(128,64),nn.Linear(64,128)),
       nn.Sequential(nn.Linear(128,64),nn.Linear(64,128))]
mli = nn.ModuleList(sli)

In [12]:
from torchinfo import summary as sm
sm(md,(4,1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
cons                                     --                        --
├─ModuleList: 1-1                        --                        --
├─Sequential: 1-2                        [4, 576]                  --
│    └─Conv2d: 2-1                       [4, 32, 26, 26]           320
│    └─MaxPool2d: 2-2                    [4, 32, 13, 13]           --
│    └─BatchNorm2d: 2-3                  [4, 32, 13, 13]           64
│    └─ReLU: 2-4                         [4, 32, 13, 13]           --
│    └─Conv2d: 2-5                       [4, 16, 12, 12]           2,064
│    └─MaxPool2d: 2-6                    [4, 16, 6, 6]             --
│    └─BatchNorm2d: 2-7                  [4, 16, 6, 6]             32
│    └─ReLU: 2-8                         [4, 16, 6, 6]             --
│    └─Flatten: 2-9                      [4, 576]                  --
├─Sequential: 1-3                        [4, 128]                  --
│    └─Line

In [13]:
md

cons(
  (front): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Conv2d(32, 16, kernel_size=(2, 2), stride=(1, 1))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Flatten(start_dim=1, end_dim=-1)
  )
  (body): Sequential(
    (0): Linear(in_features=576, out_features=256, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
  )
  (medium): Sequential(
    (

### Eval 

In [8]:
import pickle
with open("attns.pickle","wb") as fw:
    pickle.dump(evals, fw)

In [9]:
evals.update(evals_base)

In [10]:
import pandas as pd
pd.DataFrame(evals).T

,eval_loss,eval_rouge1,eval_rouge2,eval_rougeL,eval_rougeLsum,eval_gen_len,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
UFOAttention,6.409752,11.7499,1.4080,10.2117,10.2238,15.4963,39.6509,10.315,1.740,15.0
MUSEAttention,6.341923,7.2341,0.0000,6.3786,6.3928,16.3350,40.2946,10.150,1.712,15.0
SimplifiedScaledDotProductAttention,6.524034,7.5712,0.6479,6.6932,6.7240,14.6406,39.1062,10.459,1.764,15.0
ScaledDotProductAttention,7.004627,0.0000,0.0000,0.0000,0.0000,4.0000,29.5214,13.854,2.337,15.0
ExternalAttention,9.794738,0.0000,0.0000,0.0000,0.0000,5.0000,29.9376,13.662,2.305,15.0
base,8.350389,0.0000,0.0000,0.0000,0.0000,2.0000,29.3628,13.929,2.350,15.0


### basic 

In [7]:
from torch import nn
evals_base = {}
for i,attn in enumerate(['base']):

    import os
    model_name = attn
    # model_name = model_checkpoint.split("/")[-1] + model_name
    saved_path = os.path.join('./output',model_name)
    
    args = Seq2SeqTrainingArguments(
        saved_path,
        evaluation_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=15,
        predict_with_generate=True,
        fp16=True,
        save_strategy='epoch',
        load_best_model_at_end=True,
    #     push_to_hub=True,
    )

    trainer = Seq2SeqTrainer(
        model,
        args,
        train_dataset=tr,
        eval_dataset=val,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    evals[f'{model_name}'] = trainer.evaluate(eval_dataset=te)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running training *****
  Num examples = 1020
  Num Epochs = 15
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2550
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather a

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,9.611067,0.000000,0.000000,0.000000,0.000000,2.000000
2,No log,9.436999,0.000000,0.000000,0.000000,0.000000,2.000000
3,9.370300,9.270031,0.000000,0.000000,0.000000,0.000000,2.000000
4,9.370300,9.115933,0.000000,0.000000,0.000000,0.000000,2.000000
5,9.370300,8.973625,0.000000,0.000000,0.000000,0.000000,2.000000
6,8.827900,8.845672,0.000000,0.000000,0.000000,0.000000,2.000000
7,8.827900,8.732598,0.000000,0.000000,0.000000,0.000000,2.000000
8,8.827900,8.635551,0.000000,0.000000,0.000000,0.000000,2.000000
9,8.406700,8.551472,0.000000,0.000000,0.000000,0.000000,2.000000
10,8.406700,8.482576,0.000000,0.000000,0.000000,0.000000,2.000000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, id, document.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 6
Saving model checkpoint to ./output/base/checkpoint-170
Configuration saved in ./output/base/checkpoint-170/config.json
Model weights saved in ./output/base/checkpoint-170/pytorch_model.bin
tokenizer config file saved in ./output/base/checkpoint-170/tokenizer_config.json
Special tokens file saved in ./output/base/checkpoint-170/special_tokens_map.json
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/yp/anaconda3/envs/torch/lib/python3.6/site-packages/transformers/trainer.py:1357: FutureWarning: Non-finite no